In [68]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests 
from bs4 import BeautifulSoup
import time
import math
import pandas as pd
from google.colab import drive
drive.mount('drive')

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (151 kB/s)
Reading package lists... Done
Reading package lists... Done
Building

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


def timeRow():
  localtime = time.asctime( time.localtime(time.time()) )
  timeList = []
  timeList.append("updated")
  
  while localtime.find(" ") != -1:
    if localtime[0:localtime.find(" ")] != '':
      timeList.append(localtime[0:localtime.find(" ")])
    localtime = localtime[localtime.find(" ")+1:]

  last = timeList[-1]
  timeList[-1] = localtime
  localtime = last

  while localtime.find(":") != -1:
    timeList.append(localtime[0:localtime.find(":")+1])
    localtime = localtime[localtime.find(":")+1:]
  
  timeList.append(localtime)
  timeList.append('GMT+3:30')
  timeListLen = len(timeList)

  for i in range(9-timeListLen):
    timeList.append("")
  dicTime = {'State Name':timeList[0],'Percentage of Counted Votes':timeList[1],'Counted Votes for Biden':timeList[2],'Counted Votes for Trump':timeList[3],'Uncounted Votes':timeList[4],'Vote Biden Needed to Win':timeList[-4],'Percentage of Uncounted Vote Biden Needed to Win':timeList[-3],'Vote Trump Needed to Win':timeList[-2],'Percentage of Uncounted Vote Trump Needed to Win':timeList[-1]}
  return(dicTime)

firstData = True
while True:
  driver.get('https://www.google.com/search?q=us+election+results&oq=us+election+results&aqs=chrome.0.69i59j0i131i433l6j69i60.8640j0j9&sourceid=chrome&ie=UTF-8')
  timeout = 3
  try:
      element_present = EC.presence_of_element_located((By.ID, '_83GlX73aK5_G4-EPqtC9uAQ95'))
      WebDriverWait(driver, timeout).until(element_present)
  except TimeoutException:
      print("Timed out waiting for page to load")
  finally:
      print("Page loaded")
      userid_element = driver.find_elements_by_xpath('//div[@Style="width:172px;text-align:center"]')[0]
      
      userid_element = driver.find_elements_by_xpath('//span[@class="PLaf5"]')
      userid_element1 = driver.find_elements_by_xpath('//span[@class="E2WDKf"]')


      data = []
      for i in userid_element:
        if i.text != "":
          data.append(i.text)
      for i in userid_element1:
        if i.text != "":
          data.append(int(i.text.replace('% reporting','')))
      for i in range(int(len(data)/2)):
        data.append(int(driver.find_elements_by_xpath('//div[@class="oLhH0e"]')[i].find_elements_by_tag_name('span')[1].text.replace(',','')))
        data.append(int(driver.find_elements_by_xpath('//div[@class="oLhH0e"]')[i].find_elements_by_tag_name('span')[3].text.replace(',','')))



      print(data)
      rawData = []
      for i in range(int(len(data)/4)):
        d = []
        for j in range(2):
          d.append(data[i+j*6])
        d.append(data[12+i*2])
        d.append(data[13+i*2])
        rawData.append(d)
      print(rawData)

      
      info =[]
      data = rawData
      for i in range(len(data)):
        raw = data[i]
        d =[]
        d.append(raw[0])
        diff = raw[2] - raw[3]
        percent = raw[1]/100.0
        uncountedVotes = math.floor((raw[2]+raw[3])*(1-percent)/percent)
        if diff < 0:
          voteNeeded = math.ceil(((uncountedVotes + diff)/2) -diff)
        else:
          voteNeeded = math.ceil(((uncountedVotes - diff)/2))

        percentNeeded = voteNeeded/(uncountedVotes*1.0)
        d.append(str(raw[1])+"%")
        d.append(raw[2])
        d.append(raw[3])
        if percentNeeded < 1 and voteNeeded > 0:
          d.append(uncountedVotes)
          d.append(voteNeeded)
          d.append(str(math.ceil(percentNeeded*1000)/10.0)+"%")
          d.append(uncountedVotes+1-voteNeeded)
          d.append(str(math.ceil((((uncountedVotes+1-voteNeeded)/uncountedVotes)*1000))/10.0)+"%")
          info.append(d)
          print(d)
        elif voteNeeded <= 0:
          d.append(uncountedVotes)
          d.append('Already Won')
          d.append('Already Won')
          d.append('Impossible to Win in This World')
          d.append('Impossible to Win in This World')
          info.append(d)
          print(d)
        elif percentNeeded >= 1:
          d.append(uncountedVotes)
          d.append('Impossible to Win in This World')
          d.append('Impossible to Win in This World')
          d.append('Already Won')
          d.append('Already Won')
          info.append(d)
          print(d)
      if firstData == True:
        print(info)
        df = pd.DataFrame (info,columns=['State Name','Percentage of Counted Votes','Counted Votes for Biden','Counted Votes for Trump','Uncounted Votes','Vote Biden Needed to Win','Percentage of Uncounted Vote Biden Needed to Win','Vote Trump Needed to Win','Percentage of Uncounted Vote Trump Needed to Win'])
        dicTime = timeRow()
        df = df.append(dicTime, ignore_index=True)
        mainPath =r'drive/My Drive/New Folder'
        csvPath =mainPath+"/"+"USElectionSwingState.csv"
        df.to_csv(csvPath)
        oldInfo = info
        firstData = False
      else:
        flagDataChanged = False
        for i in range(len(info)):
          new = info[i]
          for j in range(len(oldInfo)):
            old = oldinfo[j]
            if new[0] == old[0]:
              for k in range(len(new)-1):
                if new[k+1] != old[k+1]:
                  flagDataChanged = True
                  print('changed '+str(old[k+1])+' to '+str(new[k+1]))
                  break
              break
          if flagDataChanged == True:
            print(info)
            df = pd.DataFrame (info,columns=['State Name','Percentage of Counted Votes','Counted Votes for Biden','Counted Votes for Trump','Uncounted Votes','Vote Biden Needed to Win','Percentage of Uncounted Vote Biden Needed to Win','Vote Trump Needed to Win','Percentage of Uncounted Vote Trump Needed to Win'],index= False)
            dicTime = timeRow()
            df = df.append(dicTime, ignore_index=True)
            mainPath =r'drive/My Drive/New Folder'
            csvPath =mainPath+"/"+"USElectionSwingState.csv"
            df.to_csv(csvPath)
            oldInfo = info
            break


      time.sleep(120)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


Timed out waiting for page to load
Page loaded
['Georgia', 'Nevada', 'North Carolina', 'Pennsylvania', 'Arizona', 'Florida', 99, 87, 99, 98, 90, 99, 2456597, 2452626, 632558, 609901, 2656207, 2732697, 3325650, 3305163, 1572346, 1535511, 5284453, 5658847]
[['Georgia', 99, 2456597, 2452626], ['Nevada', 87, 632558, 609901], ['North Carolina', 99, 2656207, 2732697], ['Pennsylvania', 98, 3325650, 3305163], ['Arizona', 90, 1572346, 1535511], ['Florida', 99, 5284453, 5658847]]
['Georgia', '99%', 2456597, 2452626, 49588, 22809, '46.0%', 26780, '54.1%']
['Nevada', '87%', 632558, 609901, 185654, 81499, '43.9%', 104156, '56.2%']
['North Carolina', '99%', 2656207, 2732697, 54433, 'Impossible to Win in This World', 'Impossible to Win in This World', 'Already Won', 'Already Won']
['Pennsylvania', '98%', 3325650, 3305163, 135322, 57418, '42.5%', 77905, '57.6%']
['Arizona', '90%', 1572346, 1535511, 345317, 154241, '44.7%', 191077, '55.4%']
['Florida', '99%', 5284453, 5658847, 110538, 'Impossible to Wi

In [87]:
def timeRow():
  localtime = time.asctime( time.localtime(time.time()) )
  timeList = []
  timeList.append("updated")
  
  while localtime.find(" ") != -1:
    if localtime[0:localtime.find(" ")] != '':
      timeList.append(localtime[0:localtime.find(" ")])
    localtime = localtime[localtime.find(" ")+1:]

  last = timeList[-1]
  timeList[-1] = localtime
  localtime = last

  while localtime.find(":") != -1:
    timeList.append(localtime[0:localtime.find(":")+1])
    localtime = localtime[localtime.find(":")+1:]
  
  timeList.append(localtime)
  timeListLen = len(timeList)

  for i in range(9-timeListLen):
    timeList.append("")
  print(timeList)
  dicTime = {'State Name':timeList[0],'Percentage of Counted Votes':timeList[1],'Counted Votes for Biden':timeList[2],'Counted Votes for Trump':timeList[3],'Uncounted Votes':timeList[4],'Vote Biden Needed to Win':timeList[-4],'Percentage of Uncounted Vote Biden Needed to Win':timeList[-3],'Vote Trump Needed to Win':timeList[-2],'Percentage of Uncounted Vote Trump Needed to Win':timeList[-1]}
  return(dicTime)

print(timeRow())

['updated', 'Sat', 'Nov', '7', '2020', '01:', '04:', '32', '']
{'State Name': 'updated', 'Percentage of Counted Votes': 'Sat', 'Counted Votes for Biden': 'Nov', 'Counted Votes for Trump': '7', 'Uncounted Votes': '2020', 'Vote Biden Needed to Win': '01:', 'Percentage of Uncounted Vote Biden Needed to Win': '04:', 'Vote Trump Needed to Win': '32', 'Percentage of Uncounted Vote Trump Needed to Win': ''}
